
The COCO object recognition model is loaded on the JetBot and an array of object indexes is created based on the image captured by the camera. 

In [1]:
from jetbot import ObjectDetector


model = ObjectDetector('../ssd_mobilenet_v2_coco.engine')

Reset the camera and run it.

In [15]:
import os

os.system('systemctl restart nvargus-daemon')

from jetbot import Camera

camera = Camera.instance(width=300, height=300)

detections = model(camera.value)

print(detections)

[[{'label': 1, 'confidence': 0.9349912405014038, 'bbox': [0.34503233432769775, 0.6683716773986816, 0.7439225912094116, 0.9909040927886963]}, {'label': 76, 'confidence': 0.9102267026901245, 'bbox': [0.009082138538360596, 0.6157286763191223, 0.3107420802116394, 1.0]}, {'label': 72, 'confidence': 0.5443169474601746, 'bbox': [0.0008737891912460327, 0.0, 0.4643605947494507, 0.5210582613945007]}, {'label': 1, 'confidence': 0.4122934937477112, 'bbox': [0.31259989738464355, 0.3512469530105591, 1.0, 0.9818780422210693]}]]


Load a widget to display detected objects.

In [3]:
from IPython.display import display
import ipywidgets.widgets as widgets




detections_widget = widgets.Textarea()

detections_widget.value = str(detections)

display(detections_widget)

Textarea(value="[[{'label': 1, 'confidence': 0.82332843542099, 'bbox': [0.007861822843551636, 0.35590872168540…

Detections: The “label” value indicates which object has been detected, the “confidence” value is the probability that the recognized object is the actual object, and the “bbox” values are the bounding box parameters of the object in the image.

In [4]:
if detections:
    image_number = 0
    object_number = 0
    print(detections[image_number][object_number])

{'label': 1, 'confidence': 0.82332843542099, 'bbox': [0.007861822843551636, 0.35590872168540955, 0.3116206228733063, 0.654403805732727]}


Load the best_path training model for collision avoidance. Preprocess camera values.

In [9]:
import torch
import torchvision
import torch.nn.functional as F
import time
import cv2
import numpy as np

collision_model = torchvision.models.alexnet(pretrained=False)
collision_model.classifier[6] = torch.nn.Linear(collision_model.classifier[6].in_features, 2)
collision_model.load_state_dict(torch.load('best_model_FreeBlocked.pth'))
device = torch.device('cuda')
collision_model = collision_model.to(device)

mean = 255.0 * np.array([0.485, 0.456, 0.406])
stdev = 255.0 * np.array([0.229, 0.224, 0.225])

normalize = torchvision.transforms.Normalize(mean, stdev)

def preprocess(camera_value):
    global device, normalize
    x = camera_value
    x = cv2.resize(x, (224, 224))
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1))
    x = torch.from_numpy(x).float()
    x = normalize(x)
    x = x.to(device)
    x = x[None, ...]
    return x

Load the JetBot controls.

Load the driving instructions based on object detection.

In [16]:
from jetbot import Robot
robot = Robot()
from jetbot import bgr8_to_jpeg
from IPython.display import display
import ipywidgets.widgets as widgets
import traitlets
blocked_widget = widgets.FloatSlider(min=0.0, max=1.0, value=0.0, description='blocked')
#image_widget = widgets.Image(format='jpeg', width=300, height=300)
label_widget = widgets.IntText(value=1, description='tracked label')

image_widget = widgets.Image(format='jpeg', width=224, height=224)  # this width and height doesn't necessarily have to match the camera

camera_link = traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)

robot.stop()

display(widgets.VBox([
    widgets.HBox([image_widget, blocked_widget]),
    label_widget
]))

width = int(image_widget.width)
height = int(image_widget.height)

def detection_center(detection):
    """Computes the center x, y coordinates of the object"""
    bbox = detection['bbox']
    center_x = (bbox[0] + bbox[2]) / 2.0 - 0.5
    center_y = (bbox[1] + bbox[3]) / 2.0 - 0.5
    return (center_x, center_y)
    
def norm(vec):
    """Computes the length of the 2D vector"""
    return np.sqrt(vec[0]**2 + vec[1]**2)

def closest_detection(detections):
    """Finds the detection closest to the image center"""
    closest_detection = None
    for det in detections:
        center = detection_center(det)
        if closest_detection is None:
            closest_detection = det
        elif norm(detection_center(det)) < norm(detection_center(closest_detection)):
            closest_detection = det
    return closest_detection

# Simple PD controller (Kp - proportional term, Kd - derivative term)

Kp = 0.18
Kd = 0.05

def update(change):
    global robot, frame_counter, n_frames_stuck
    x = change['new'] 
    x = preprocess(x)
    y = model(x)
    
    # apply the `softmax` function to normalize the output vector so it sums to 1 (which makes it a probability distribution)
    y = F.softmax(y, dim=1)
    
    y = y.flatten()
          
def execute(change):
    image = change['new']
    
    # execute collision model to determine if blocked
    collision_output = collision_model(preprocess(image)).detach().cpu()
    prob_blocked = float(F.softmax(collision_output.flatten(), dim=0)[0]) # probability of blocked, first dataset category
    prob_caution = float(F.softmax(collision_output.flatten(), dim=0)[1]) # probability of caution, second dataset category
    prob_free = float(F.softmax(collision_output.flatten(), dim=0)[2]) # probability of free, third dataset category
    blocked_widget.value = prob_blocked
    
    min_drive = .5
    min_blocked = .6
    min_turn = .5                         # Turn slow
    max_turn = .7                         # Turn fast
    frwd_value = 0.6                      # Default value to drive forward (0 = no action, 1 = full motor capacity)
    frwd_caution = 0.4                    # Caution speed
    bkwd_value = 0.5                      # Default reverse value
    min_free = .40    # Min free for robot to drive forward 
    max_n_frames_stuck = 10               # Limit on the number of frames the robot is stuck for. Once this limit is reached, robot goes in reverse
    frame_counter = 0                     # Frame counter 
    n_frames_stuck = 0                    # Initialize counter of the number of successive frames the robot is stuck for  
    data_log = []                         # Initialize the array whcih will store a history of telemetry readings and robot actions (for analysis and tuning) 
    
    # define functions which determine (and return) robot actions
    def forward(value):
        robot.forward(value)
        return ("FWRD",round(value,2))

    def left(value):
        robot.left(value)
        return ("LEFT",round(value,2))

    def right(value):
        robot.right(value)
        return ("RIGHT",round(value,2))
    
    def reverse(value):
        robot.backward(value)
        return ("BKWRD",round(value,2))
    
    action = ""
            
    # compute all detected objects
    detections = model(image)

    # draw a box around detection on image
    for det in detections[0]:
        bbox = det['bbox']
        cv2.rectangle(image, (int(width * bbox[0]), int(height * bbox[1])), (int(width * bbox[2]), int(height * bbox[3])), (255, 0, 0), 2)
        

    # select detections that match selected class label
    matching_detections = [d for d in detections[0] if d['label'] == int(label_widget.value)]
    # get detection closest to center of field of view and draw it
    det = closest_detection(matching_detections) 
    
    detectionList = []
    
    if det: # true if there are any detections
            bbox = det['bbox']
            cv2.rectangle(image, (int(width * bbox[0]), int(height * bbox[1])), (int(width * bbox[2]), int(height * bbox[3])), (0, 255, 0), 5)          
            
            # move robot forward and steer proportional target's x-distance from center
            center = detection_center(det)
            robot.set_motors(
                float(frwd_value + min_turn * center[0]),
                float(frwd_value - min_turn * center[0])
            )
            image_widget.value = bgr8_to_jpeg(image)
            detectionList = detectionList.append(det)
    if prob_free < prob_caution or prob_blocked:
        if prob_caution < prob_blocked:
            if n_frames_stuck < max_n_frames_stuck: 
                action = right(max_turn)
                n_frames_stuck = n_frames_stuck + 1
                image_widget.value = bgr8_to_jpeg(image)
                return
            else:
                action = reverse(bkwd_value)
                n_frames_stuck = 0

        else:
            action = right(min_turn)
            action = frwd_caution
            n_frames_stuck = n_frames_stuck + 1
            return
    else:
        robot.forward(frwd_value)

execute({'new': camera.value}) 

Connect the update function to each camera update

In [17]:
camera.unobserve_all()
camera.observe(execute, names='value')

End processing and stop the robot

In [18]:
camera.unobserve_all()
time.sleep(1.0)
robot.stop()